# RAG DEMO

## Loading files

### pdf basic parsing

In [16]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/inputs/cours_de_philosophie.pdf"

In [17]:
loader_local = PyPDFLoader(file_path)
pages = loader_local.load()

In [ ]:
pages

In [21]:
print(pages[100].page_content)

 101
posséder sa mère et tuer son père qui  a contra int Freud à inventer l’inconscient psychique et 
ses mécanismes. 
 
    Ainsi Girard prétend-il s’appuyer à la fois sur l’intuition de Freud selon laquelle le désir est 
le propre de l’homme et sur celle d’Aristote qui affirme que «  l’homme  diffère des autres 
animaux en ce qu’il est le plus apte à l’im itation », afin de développer sa conception 
concernant le caractère mimétique du désir. 
 
 
III Le désir ultime de l’homme : être heureux  (L, ES, S, Tech.) 
 
L’homme est un être de désir, c’est une affaire entendue. Ces désirs sont successifs et sans 
répit. Je peux désirer faire des études afin d’avoir un bon métier ; je désire avoir un bon métier 
en vue d’accéder à des conditions de vie confortables etc. Tout désir peut s’avérer un moyen 
en vue d’autre chose. Mais il semble qu’il y ait un désir qui ne  soit jamais un moyen en vue 
d’autre chose et ce désir c’est le désir d’être heureux. En vue de quoi un homme veut-il être 


## Chunking

This code demonstrates how to split a collection of documents (`pages`) into smaller chunks using `RecursiveCharacterTextSplitter` from the `langchain_text_splitters` library. The parameters used are:

- `chunk_size=1000`: The maximum length of each text chunk.
- `chunk_overlap=100`: The number of overlapping characters between consecutive chunks.

The output, `splits`, is a list of the resulting text chunks, and the length of this list is displayed to indicate the total number of chunks generated.

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(pages)
len(splits)



1904

## Embedding + Indexing

This code sets up embeddings and a vector store for working with text data using the `MistralAI` and `Chroma` libraries:

1. **Environment Setup**:
   - The `dotenv` library is used to load environment variables from a `.env` file.
   - The `MISTRAL_API_KEY` is retrieved using `os.getenv`.

2. **MistralAI Embeddings**:
   - The `MistralAIEmbeddings` class is initialized with the `mistral-embed` model and the provided API key.
   - These embeddings can be used for tasks such as similarity search or semantic clustering.
   
The embeddings object will serve as the foundation for generating vector representations of text data.

In [25]:
from langchain_chroma import Chroma
from langchain_mistralai import MistralAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")


embeddings = MistralAIEmbeddings(model="mistral-embed", api_key=api_key)

/Users/clementdesroches/Documents/GitHub/rag-demo/.venv/lib/python3.9/site-packages/langchain_mistralai/embeddings.py:175: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [26]:
query_result = embeddings.embed_query("hello")
query_result

[-0.0267333984375,
 -0.0106353759765625,
 0.006134033203125,
 0.018829345703125,
 0.05316162109375,
 0.04150390625,
 0.038726806640625,
 0.00778961181640625,
 -0.02435302734375,
 -0.00856781005859375,
 -0.046478271484375,
 0.022796630859375,
 -0.02764892578125,
 -0.020843505859375,
 -0.0310821533203125,
 -0.0058746337890625,
 -0.013916015625,
 0.048431396484375,
 0.02740478515625,
 0.050201416015625,
 -0.0038166046142578125,
 -0.01032257080078125,
 -0.03753662109375,
 0.00762939453125,
 -0.0338134765625,
 0.00960540771484375,
 -0.0090484619140625,
 -0.0250244140625,
 -0.0054168701171875,
 -0.00756072998046875,
 0.0011873245239257812,
 -0.0226898193359375,
 0.029388427734375,
 -0.0269012451171875,
 0.03076171875,
 -0.0112762451171875,
 -0.0386962890625,
 -0.036041259765625,
 0.03948974609375,
 0.0255126953125,
 -0.0080413818359375,
 -0.0279693603515625,
 0.0008654594421386719,
 6.365776062011719e-05,
 0.0209808349609375,
 -0.01702880859375,
 0.01224517822265625,
 -0.028289794921875,
 0.

## Indexing in a Chroma database

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="philo",
    embedding_function=embeddings,
    persist_directory="../data/chroma_langchain_db",  # Where to save data locally, remove if not necessary
)


In [28]:
from uuid import uuid4
from langchain_community.vectorstores.utils import filter_complex_metadata

docs = filter_complex_metadata(splits)
uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

['2eb98fd7-e4f1-4a24-9660-ceb0b89a1efc',
 '49a3939d-c029-437c-80a7-176474f9def9',
 'c8009f1f-c578-454d-a0c1-90b06708a73f',
 'a4ce6516-86da-4ae2-bf08-02810e9324ac',
 '85f021ab-b997-4dca-b97e-ce919eddd311',
 '7d4b6fc7-9750-4e9d-9a5a-5104b55f6048',
 'a1a539b7-8318-4ed2-84fb-de7e7e19e84d',
 'dbf1061c-1263-4b0e-af75-1271787b2ba4',
 '4645bcd7-5fe5-4c98-8a08-af17ffe493b2',
 '0106b58e-5a77-48f9-a164-792641990655',
 '3dfe6d6a-2960-45a7-8f3c-18af4398bf25',
 '6f9bf9ad-6616-47dc-bf84-6a49c0e4e9e1',
 'e19bc70c-ef42-4169-9adb-0787a17b5bc0',
 '18141c2a-fe13-4103-8ffa-5ec608dee5bb',
 '0304baf0-0e81-4b27-910e-7d05e535a7c4',
 '0266afa5-1dd4-4cb1-8685-1217ac7c9a9a',
 '0b60683c-fdd6-4531-a4b6-1935d5fbcc9f',
 '90f4244b-1009-4fd2-a8a5-2d5c4ae95a24',
 '853a55c1-154a-4499-8b04-e2859d585d15',
 '86a1bff8-a2c1-4f91-afad-a2e5c9f0e988',
 '8c5f237b-f2c9-4858-a633-1f669eb9ac03',
 '1673358b-aa14-45e9-a4e4-f958349716fc',
 'e2d4735f-9436-478b-9c4a-89dfb8e93e3f',
 'b826cc42-2ddb-4201-99a2-8b96b0ef27a7',
 'e9b3f82a-248d-

## Similarity search

This code performs a semantic similarity search using a vector store, aiming to retrieve documents most relevant to the query

1. **Similarity Search**:
   - The method `similarity_search_with_score` retrieves the top `k=4` most similar documents along with their similarity scores.

2. **Output Formatting**:
   - For each result, the document's content (`res.page_content`) and metadata (`res.metadata`) are displayed.
   - The similarity score (`score`) is formatted with three decimal places and is used to indicate the relevance of each document to the query.

This approach is useful for querying knowledge bases or retrieving relevant passages in natural language processing tasks.

In [ ]:
results = vector_store.similarity_search_with_score(
    "Quelle est la position de Freud sur l'inconscient ?", k=4
)
for res, score in results:
    print(f"\n\n* [SIM={score:3f}] {res.page_content} [{res.metadata}]")



* [SIM=0.251413] 74
L’apport spécifique de Freud : le rôle de la sexualité dans l’inconscient psychique 
 
Comme on peut le constater,  la découverte d’un inconscient psychique n’est pas une 
découverte de Freud, contrairement à ce que beaucoup pensent encore de nos jours. 
L’inconscient était déjà connu par la psychologie de la fin du XIX° siècle. L’apport de Freud 
se situe  ailleurs. En effet, au contact de ses patients, Freud croit  constater le rôle décisif de la 
sexualité dans la formation  et le contenu de l’ inconscient psychique . Cette affirmation a 
conduit à développer nombre d’inco mpréhensions et de résistances  vis-à-vis des théories de 
Freud.  
 
En effet, l’opinion commune conçoit la sexu alité comme l’ensemble des comportements  
relatifs aux relations intimes et donc de nature génitale entre adultes ou  bien concernant les 
plaisirs solitaires de même type. Freud pour sa part attribue à ce terme à la fois un sens [{'page': 73, 'source': '../data/inputs/cours_de_p

In [32]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

retrieved_docs = retriever.invoke("Les pratiques artistiques transforment-elles le monde?")

len(retrieved_docs)

4

In [33]:
print(retrieved_docs[0].page_content)

dans certains cas, pour en avoi r entendu parler par des personne s de nos connaissances ? On 
peut même dire que ces efforts inutiles se réduisent à un jeu présomptueux dont les résultats 
restent toujours inférieurs à ce  que nous offre la nature. C’es t que l’art, limité dans ses 
moyens d’expression, ne peut produire que des illusion s unilatérales, offrir l’apparence de la 
réalité à un seul de nos se ns ; et, en fait, lorsqu’il ne va pas au-delà de la simple imitation, il 
est incapable de nous donner l’impression d’une r éalité vivante ou d’une vie réelle : tout ce 
qu’il peut nous offrir, c’est une caricature de la vie. 
Hegel 
 
Questions 
1 Dégagez les articulations du texte ainsi que son idée principale. 
2 Expliquez : « l’art… ne peut nous offrir qu’une caricature de la vie » 
3 Une œuvre d’art nous invite-t-elle à nous évader du monde ? 
  
La philosophie n’est pas l’art, mais elle a avec l’art de profondes affinités. Qu’est-ce que


## Using GenAI to answer

### Prompt template

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)


### Answer

In [43]:
query = "Quelle est la position de Freud sur l'inconscient ?"
retrieved_docs = retriever.invoke(query)
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])


In [45]:
print(context_text)

74
L’apport spécifique de Freud : le rôle de la sexualité dans l’inconscient psychique 
 
Comme on peut le constater,  la découverte d’un inconscient psychique n’est pas une 
découverte de Freud, contrairement à ce que beaucoup pensent encore de nos jours. 
L’inconscient était déjà connu par la psychologie de la fin du XIX° siècle. L’apport de Freud 
se situe  ailleurs. En effet, au contact de ses patients, Freud croit  constater le rôle décisif de la 
sexualité dans la formation  et le contenu de l’ inconscient psychique . Cette affirmation a 
conduit à développer nombre d’inco mpréhensions et de résistances  vis-à-vis des théories de 
Freud.  
 
En effet, l’opinion commune conçoit la sexu alité comme l’ensemble des comportements  
relatifs aux relations intimes et donc de nature génitale entre adultes ou  bien concernant les 
plaisirs solitaires de même type. Freud pour sa part attribue à ce terme à la fois un sens

notre vie intérieure profonde et méritent à ce titre d’être déchiffr

In [47]:
prompt = prompt_template.format(context=context_text, question=query)
print(prompt)

Human: 
Answer the question based only on the following context:
74
L’apport spécifique de Freud : le rôle de la sexualité dans l’inconscient psychique 
 
Comme on peut le constater,  la découverte d’un inconscient psychique n’est pas une 
découverte de Freud, contrairement à ce que beaucoup pensent encore de nos jours. 
L’inconscient était déjà connu par la psychologie de la fin du XIX° siècle. L’apport de Freud 
se situe  ailleurs. En effet, au contact de ses patients, Freud croit  constater le rôle décisif de la 
sexualité dans la formation  et le contenu de l’ inconscient psychique . Cette affirmation a 
conduit à développer nombre d’inco mpréhensions et de résistances  vis-à-vis des théories de 
Freud.  
 
En effet, l’opinion commune conçoit la sexu alité comme l’ensemble des comportements  
relatifs aux relations intimes et donc de nature génitale entre adultes ou  bien concernant les 
plaisirs solitaires de même type. Freud pour sa part attribue à ce terme à la fois un sens

not

In [51]:
response_text = llm.invoke(prompt)

In [52]:
print(response_text)

content="Freud soutient que l'inconscient psychique est une réalité nécessaire pour expliquer les actes psychiques qui ne bénéficient pas des témoignages de la conscience. Il affirme que les données de la conscience sont extrêmement lacunaires et que des actes psychiques fréquents présupposent d'autres actes qui échappent à la conscience. Selon Freud, l'inconscient psychique n'est pas une simple suspension de la conscience ni un manque de réflexion, mais une forme de psychisme qui échappe à l'investigation consciente. Il défend l'idée que l'hypothèse de l'inconscient est légitime et nécessaire pour comprendre la vie intérieure profonde et les motivations véritables des actes humains." additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1336, 'total_tokens': 1528, 'completion_tokens': 192}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'} id='run-29978c57-e497-4117-b32b-63f682f2701a-0' usage_metadata={'input_tokens': 1336, 'output_tokens': 192, 'total_token